In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from shiny.express import input, render, ui
from shinywidgets import render_widget
from shiny import run_app
import threading
from shiny import App, ui, render, reactive, session, run_app



In [2]:
NFLVERSE_DATA_PATH = r"C:\Users\schne\Desktop\nfldata\player_stats.csv"
NFLVERSE_TEAMS_DATA_PATH = r"C:\Users\schne\Desktop\nfldata\team_stats.csv"
NFLVERSE_INJURIES_PATH = r"C:\Users\schne\Desktop\nfldata\nfl_injuries.xlsx"
NFLVERSE_DEPTH_CHART_PATH = r"C:\Users\schne\Desktop\nfldata\depth.csv"
NFLVERSE_COMBINE_PATH = r"C:\Users\schne\Desktop\nfldata\combine_data.csv"
NFLVERSE_DRAFT_PICKS_PATH = r"C:\Users\schne\Desktop\nfldata\draft_picks.csv"
NFLVERSE_HISTORICAL_CONTRACTS_PATH = r"C:\Users\schne\Desktop\nfldata\historical_contracts.csv"
NFLVERSE_PLAYERS_INFO_PATH = r"C:\Users\schne\Desktop\nfldata\player_info.csv"
NFLVERSE_SCHEDULE_PATH = r"C:\Users\schne\Desktop\nfldata\schedule.csv"
NFLVERSE_SNAP_COUNT_PATH = r"C:\Users\schne\Desktop\nfldata\snap_count.csv"
NFLVERSE_TEAM_STATS_PATH = r"C:\Users\schne\Desktop\nfldata\team_stats.csv"
NFLVERSE_TEAMS_INFO_PATH = r"C:\Users\schne\Desktop\nfldata\teams_info.csv"
NFLVERSE_TRADE_INFO_PATH = r"C:\Users\schne\Desktop\nfldata\trade_info.csv"
NFLVERSE_WEEKLY_FF_STATS_PATH = r"C:\Users\schne\Desktop\nfldata\week_ff.csv"


all_players_df = pd.read_csv(NFLVERSE_DATA_PATH)
all_teams_df = pd.read_csv(NFLVERSE_TEAMS_DATA_PATH)
injuries_df = pd.read_excel(NFLVERSE_INJURIES_PATH, engine='openpyxl')
depth_df = pd.read_csv(NFLVERSE_DEPTH_CHART_PATH)
combine_df = pd.read_csv(NFLVERSE_COMBINE_PATH)
draft_picks_df = pd.read_csv(NFLVERSE_DRAFT_PICKS_PATH)
historical_contracts_df = pd.read_csv(NFLVERSE_HISTORICAL_CONTRACTS_PATH)
players_info_df = pd.read_csv(NFLVERSE_PLAYERS_INFO_PATH)
schedule_df = pd.read_csv(NFLVERSE_SCHEDULE_PATH)
snap_count_df = pd.read_csv(NFLVERSE_SNAP_COUNT_PATH)
team_stats_df = pd.read_csv(NFLVERSE_TEAM_STATS_PATH)
teams_info_df = pd.read_csv(NFLVERSE_TEAMS_INFO_PATH)
trade_info_df = pd.read_csv(NFLVERSE_TRADE_INFO_PATH)
weekly_ff_df = pd.read_csv(NFLVERSE_WEEKLY_FF_STATS_PATH)

C:\Users\schne\AppData\Local\Temp\ipykernel_18232\1895870124.py:17: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_df = pd.read_csv(NFLVERSE_DATA_PATH)
C:\Users\schne\AppData\Local\Temp\ipykernel_18232\1895870124.py:20: DtypeWarning: Columns (1,3,5,6,7,8,11,12,13,14,15,16,17,20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  depth_df = pd.read_csv(NFLVERSE_DEPTH_CHART_PATH)


In [3]:
column_categories = {
    "standard": {
        "player_id",
        "player_name",
        "player_display_name",
        "position",
        "position_group",
        "season",
        "week",
        "season_type",
        "team",
        "opponent_team",
        "pacr",
        #"report_status_Doubtful",
        #"report_status_Note",
        #"report_status_Out",
        #"report_status_Probable",
        #"report_status_Questionable",
        #"report_status_nan",
        #"practice_status_\n    ",
        #"practice_status_Did Not Participate In Practice",
        #"practice_status_Full Participation in Practice",
        #"practice_status_Limited Participation in Practice",
        #"practice_status_Note",
        #"practice_status_Out (Definitely Will Not Play)",
        #"depth_team", 
    },
    "passing": {
        "completions",
        "attempts",
        "passing_yards",
        "passing_tds",
        "passing_interceptions",
        "sacks_suffered",
        "sack_yards_lost",
        "sack_fumbles",
        "sack_fumbles_lost",
        "passing_air_yards",
        "passing_yards_after_catch",
        "passing_first_downs",
        "passing_epa",
        "passing_cpoe",
        "passing_2pt_conversions",
        "carries", # Include rushing stats for QBs
        "rushing_yards",
        "rushing_tds",
        "rushing_fumbles",
        "rushing_fumbles_lost",
        "rushing_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
    },
    "rushing_and_receiving": {
        "carries",
        "rushing_yards",
        "rushing_tds",
        "rushing_fumbles",
        "rushing_fumbles_lost",
        "rushing_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
        "receptions",
        "targets",
        "receiving_yards",
        "receiving_tds",
        "receiving_fumbles",
        "receiving_fumbles_lost",
        "receiving_air_yards",
        "receiving_yards_after_catch",
        "receiving_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
        "receptions",
        "targets",
        "receiving_yards",
        "receiving_tds",
        "receiving_fumbles",
        "receiving_fumbles_lost",
        "receiving_air_yards",
        "receiving_yards_after_catch",
        "receiving_first_downs",
        "receiving_epa",
        "receiving_2pt_conversions",
        "racr",
        "target_share",
        "air_yards_share",
        "wopr",
    },
    "defense": {
        "special_teams_tds",
        "def_tackles_solo",
        "def_tackles_with_assist",
        "def_tackle_assists",
        "def_tackles_for_loss",
        "def_tackles_for_loss_yards",
        "def_fumbles_forced",
        "def_sacks",
        "def_sack_yards",
        "def_qb_hits",
        "def_interceptions",
        "def_interception_yards",
        "def_pass_defended",
        "def_tds",
        "def_fumbles",
        "def_safeties",
        "misc_yards",
        "fumble_recovery_own",
        "fumble_recovery_yards_own",
        "fumble_recovery_opp",
        "fumble_recovery_yards_opp",
        "fumble_recovery_tds",
        "penalties",
    },
    "kicking": {
        "fg_made",
        "fg_att",
        "fg_missed",
        "fg_blocked",
        "fg_long",
        "fg_pct",
        "fg_made_0_19",
        "fg_made_20_29",
        "fg_made_30_39",
        "fg_made_40_49",
        "fg_made_50_59",
        "fg_made_60_",
        "fg_missed_0_19",
        "fg_missed_20_29",
        "fg_missed_30_39",
        "fg_missed_40_49",
        "fg_missed_50_59",
        "fg_missed_60_",
        "fg_made_list",
        "fg_missed_list",
        "fg_blocked_list",
        "fg_made_distance",
        "fg_missed_distance",
        "fg_blocked_distance",
        "pat_made",
        "pat_att",
        "pat_missed",
        "pat_blocked",
        "pat_pct",
        "gwfg_made",
        "gwfg_att",
        "gwfg_missed",
        "gwfg_blocked",
        "gwfg_distance",
    },
    "special_teams": {
        "punt_returns",
        "punt_return_yards",
        "kickoff_returns",
        "kickoff_return_yards",
    },
}

categories_positions = {
    "passing": ["QB"],
    "rushing_and_receiving": ["RB", "WR"],
    "kicking": ["K"] 
}


In [4]:

# Establish dataframes for each positional group, filter by position value
def filter_by_positional_group(df, category_key_a) -> pd.DataFrame:
    """
    Filters the DataFrame for a given category and relevant positions,
    always including the standard columns (player_name, team, position).
    """
    # Ensure 'standard' exists
    standard_cols = column_categories.get("standard", set())
    
    # Combine standard columns with category-specific columns
    category_cols = column_categories.get(category_key_a, set())
    cols_to_keep = standard_cols | category_cols
    
    # Select columns
    new_df = df.loc[:, list(cols_to_keep)]
    
    # Filter rows by positions for this category
    positions = categories_positions.get(category_key_a, [])
    new_df = new_df[new_df["position"].isin(positions)]
    
    # Reset index
    return new_df.reset_index(drop=True)

# Data of interest
passing_df = filter_by_positional_group(all_players_df, "passing")
rushing_and_receiving_df = filter_by_positional_group(all_players_df, "rushing_and_receiving")

In [5]:
pd.set_option('display.max_columns', None) 
passing_df.head()

,sack_fumbles_lost,passing_tds,season_type,passing_air_yards,rushing_fumbles,team,passing_yards,rushing_epa,sack_fumbles,week,rushing_first_downs,passing_interceptions,player_id,passing_2pt_conversions,rushing_fumbles_lost,passing_epa,rushing_2pt_conversions,player_name,sacks_suffered,completions,passing_yards_after_catch,passing_first_downs,season,carries,passing_cpoe,position_group,rushing_yards,opponent_team,pacr,rushing_tds,attempts,position,player_display_name,sack_yards_lost
0,0,5,REG,39,0,DAL,362,1.451270,0,1,1,3,00-0000104,0,0,12.898533,0,T.Aikman,1,28,362,15,1999,2,NaN,QB,4,WAS,9.282051,0,49,QB,Troy Aikman,-7
1,0,3,REG,3,0,DET,216,-2.841205,0,1,0,1,00-0000865,0,0,-3.817581,0,C.Batch,5,16,216,10,1999,3,NaN,QB,-3,SEA,72.000000,0,26,QB,Charlie Batch,-14
2,1,1,REG,4,0,CAR,207,0.659793,1,1,0,1,00-0001218,0,0,-17.287012,0,S.Beuerlein,7,16,207,9,1999,1,NaN,QB,8,NO,51.750000,0,32,QB,Steve Beuerlein,-46
3,0,2,REG,29,1,CIN,182,7.742478,0,1,7,2,00-0001335,2,0,10.701178,0,J.Blake,1,18,182,13,1999,11,NaN,QB,90,TEN,6.275862,0,31,QB,Jeff Blake,0
4,1,1,REG,7,0,NE,340,-0.411763,1,1,0,1,00-0001361,0,0,6.823221,0,D.Bledsoe,2,21,340,13,1999,1,NaN,QB,-1,NYJ,48.571429,0,30,QB,Drew Bledsoe,-14


In [6]:
# Filter for 2024 and 2025 seasons
passing_df = passing_df[passing_df['season'].isin([2024, 2025])]
passing_df_data = passing_df
#player_name = "Aaron Rodgers"
#passing_df = passing_df[passing_df['player_display_name'] == player_name]

# Get unique player names and sort
players = sorted(passing_df['player_display_name'].dropna().unique().tolist())



In [7]:
passing_df_data = passing_df_data.rename(columns={"player_display_name": "player"})
cols =["player"] + [col for col in passing_df_data.columns if col != "player"]
passing_df_data = passing_df_data[cols]
#passing_df_data = passing_df_data.iloc[:,:4]

passing_df_data.head()

,player,sack_fumbles_lost,passing_tds,season_type,passing_air_yards,rushing_fumbles,team,passing_yards,rushing_epa,sack_fumbles,week,rushing_first_downs,passing_interceptions,player_id,passing_2pt_conversions,rushing_fumbles_lost,passing_epa,rushing_2pt_conversions,player_name,sacks_suffered,completions,passing_yards_after_catch,passing_first_downs,season,carries,passing_cpoe,position_group,rushing_yards,opponent_team,pacr,rushing_tds,attempts,position,sack_yards_lost
15989,Aaron Rodgers,0,1,REG,164,0,NYJ,167,0.000000,0,1,0,1,00-0023459,0,0,3.258283,0,A.Rodgers,1,13,93,8,2024,1,-5.897989,QB,-1,SF,1.018293,0,21,QB,-5
15990,Matthew Stafford,0,1,REG,290,0,LA,317,NaN,0,1,0,1,00-0026498,0,0,4.536421,0,M.Stafford,2,34,171,15,2024,0,0.365350,QB,0,DET,1.093103,0,49,QB,-13
15991,Andy Dalton,0,0,REG,6,0,CAR,0,NaN,0,1,0,0,00-0027973,0,0,-0.621024,0,A.Dalton,0,0,0,0,2024,0,-71.000129,QB,0,NO,0.000000,0,1,QB,0
15992,Tyrod Taylor,0,1,REG,31,0,NYJ,36,0.772778,0,1,0,0,00-0028118,0,0,2.263537,0,T.Taylor,0,6,24,3,2024,1,7.990048,QB,7,SF,1.161290,0,8,QB,0
15993,Kirk Cousins,0,1,REG,140,0,ATL,155,-5.980317,0,1,1,2,00-0029604,0,0,-11.084853,0,K.Cousins,2,16,73,8,2024,1,-1.985487,QB,0,PIT,1.107143,0,26,QB,-18


In [8]:


app_ui = ui.page_fluid(
        ui.input_selectize("player", "Select Player:", choices=players, selected = players[0]),
        ui.h2("Passing Statistics"),
        ui.output_data_frame("passing_table"),
)


def server(input, output, session):
    
    @output
    @render.data_frame
    def passing_table():
        selected = input.player()
        if selected:
            df = passing_df_data[passing_df_data["player"] == selected]
        else:
            df = passing_df_data
        return render.DataGrid(df)

app = App(app_ui, server)


In [ ]:
import threading
from shiny import run_app

# Run the Shiny app in a background thread
def run_shiny():
    run_app(app, port=8000)  # Don't use host in notebook

thread = threading.Thread(target=run_shiny, daemon=True)
thread.start()

print("Shiny app should now be running in your browser at http://127.0.0.1:8000")




Shiny app should now be running in your browser at http://127.0.0.1:8000


INFO:     Started server process [18232]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51545 - "GET / HTTP/1.1" 200 OK


INFO:     127.0.0.1:51549 - "WebSocket /websocket/" [accepted]
INFO:     connection open


In [10]:

app_ui = ui.page_sidebar(
    ui.sidebar(
        ui.input_selectize("player", "Select Player:", choices=players, selected = "Aaron Rodgers"),
        ),
)
ui.layout_columns(
    ui.card(
        ui.card_header("Passing Statistics"),
        ui.output_data_frame("passing_df"),
    ),
)
def server(input, output, session):

    @reactive.calc
    def get_player():
        selected = input.player()
        return passing_df[passing_df["player_display_name"] == selected]

    @render.data_frame
    def passing_df():
        data = get_player().set_index("player_display_name").T
        result = pd.DataFrame({
            "Category": data.index,
            "Value": data.values.flatten()
        })
        result["Value"] = result["Value"].apply(lambda v: f"{v:.1f}" if isinstance(v, (int, float)) else v)
        return render.DataGrid(result, selection_mode="row")

In [11]:
from shiny import App, ui, render, reactive, session
import pandas as pd

# Example dataframe (replace with your all_players_df)


# UI
# --- UI ---
app_ui = ui.page_fluid(
    ui.input_select("player", "Select Player:", sorted(passing_df['player_display_name'].unique())),
    "the shape of the dataframe is:",
    ui.output_code("passing_df_shape"),
)



# --- Server ---
def server(input, output, session):

    @render.code
    def 
    
    @render.data_frame
    def passing_df():
        return render.DataTable(passing_df, filters= True)
    
    @reactive.effect
    @reactive.data.frame
    def player_stats():
        selected_player = input.player()
        if not selected_player:
            return pd.DataFrame({"Message": ["Select a player to see stats"]})
        
        # Filter dataframe for the selected player
        df = passing_df[passing_df['player_display_name'] == selected_player]
        return render.DataTable(df, filters= True) #df.reset_index(drop=True)
    
    @render.code
    def passing_df_shape():
        return passing_df.shape

app = App(app_ui, server)

SyntaxError: invalid syntax (805985475.py, line 21)

In [ ]:
from shiny import App, ui, render, reactive, session
import pandas as pd

# Example dataframe (replace with your all_players_df)


# UI
app_ui = ui.page_fluid(
    ui.h2("NFL Player Fantasy Points Dashboard"),
    
    ui.navset_tab(
        ui.nav_panel(
            "Player Stats Table",
            ui.h4("Step 1: Select a Position"),
            ui.input_select("position", "Position:", positions),
            
            ui.h4("Step 2: Select a Player"),
            ui.input_select("player", "Player:", choices=[]),  # Will be updated dynamically
            
            ui.h4("Player Stats"),
            ui.output_table("stats_table")
        )
    )
)

# Server
def server(input, output, session):
    
    # Reactive: Filter players based on position
    @reactive.Calc
    def filtered_players():
        pos = input.position()
        players = all_players_df[all_players_df['position'] == pos]['players'].dropna().tolist()
        players.sort()
        return players
    
    # Update player dropdown when position changes
    @reactive.Effect
    def update_player_dropdown():
        players = filtered_players()
        session.set_input_choices("player", players)
        if players:
            session.set_input_value("player", players[0])
    
    # Render the table
    @output()
    @render.table
    def stats_table():
        selected_player = input.player()
        if not selected_player:
            return pd.DataFrame({"Message": ["Select a player to see stats"]})
        
        # Filter dataframe for the selected player
        df = all_players_df[all_players_df['player'] == selected_player]
        # Optionally, drop columns you don't want
        df = df.reset_index(drop=True)
        return df

app = App(app_ui, server)


In [ ]:
# Define UI with correct sidebar layout
app_ui = ui.page_fluid(
    ui.layout_sidebar(
        ui.sidebar_panel(
            ui.h2("NFL Player Stats"),
            ui.input_select("position", "Select Position:", sorted(all_players_df['position'].dropna().unique())),
            ui.input_select("player", "Select Player:", choices=[])
        ),
        ui.main_panel(
            ui.h3("Player Stats Table"),
            ui.output_table("player_table"),
            ui.h3("Player Stats Over Season"),
            ui.output_plot("player_plot")
        )
    )
)

# Server logic
def server(input, output, session):

    # Update player dropdown based on position
    @reactive.Effect
    @reactive.event(input.position)
    def update_player_choices():
        filtered_players = all_players_df[all_players_df['position'] == input.position()]
        session.set_input_choices("player", filtered_players['player_name'].unique())

    # Filtered player data
    @reactive.Calc
    def player_data():
        if input.player() is None:
            return pd.DataFrame()
        return all_players_df[(all_players_df['position'] == input.position()) &
                              (all_players_df['player_name'] == input.player())]

    # Render table
    @output
    @render.table
    def player_table():
        df = player_data()
        return df if not df.empty else pd.DataFrame({"Message": ["No data available"]})

    # Render plot
    @output
    @render.plot
    def player_plot():
        df = player_data()
        if df.empty:
            return px.line(title="No data to display")
        # Example: Plot passing_yards, rushing_yards, receiving_yards if they exist
        stats_columns = [col for col in ['passing_yards', 'rushing_yards', 'receiving_yards'] if col in df.columns]
        if not stats_columns:
            return px.line(title="No relevant stats columns found")
        df_melted = df.melt(id_vars=['week'], value_vars=stats_columns, var_name='Stat', value_name='Value')
        fig = px.line(df_melted, x='week', y='Value', color='Stat', markers=True,
                      title=f"{input.player()} Stats Over Season")
        return fig

# Create app
app = App(app_ui, server)